#Python para Investimentos

*   Base de Dados: http://dados.cvm.gov.br/




###Bibliotecas

In [1]:
import pandas as pd
from pandas.tseries.offsets import BDay
from datetime import date
pd.set_option("display.max_colwidth", 150)

###Funções para Coleta e Tratamento de dados de Fundos de Investimento fornecidos pela CVM

In [2]:
def busca_cadastro_cvm(data=(date.today()-BDay(144))):
  if data is not busca_cadastro_cvm.__defaults__[0]:
    data = pd.to_datetime(data)
  
  try:
    url = 'http://dados.cvm.gov.br/dados/FI/CAD/DADOS/inf_cadastral_fi_{}{:02d}{:02d}.csv'.format(data.year, data.month, data.day)
    return pd.read_csv(url, sep=';', encoding='ISO-8859-1')

  except: 
    print("Arquivo {} não encontrado!".format(url))
    print("Forneça outra data!")

In [3]:
def busca_informes_diarios_cvm_por_periodo(data_inicio, data_fim):
  datas = pd.date_range(data_inicio, data_fim, freq='MS') 
  informe_completo = pd.DataFrame()

  for data in datas:
    try:
      url ='http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{:02d}.csv'.format(data.year, data.month)
      informe_mensal = pd.read_csv(url, sep=';')    
    
    except: 
      print("Arquivo {} não encontrado!".format(url))    

    informe_completo = pd.concat([informe_completo, informe_mensal], ignore_index=True)

  return informe_completo

In [4]:
def melhores_e_piores(informes, cadastro, top=5, minimo_de_cotistas=100, classe=''):  
  cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes['NR_COTST'] >= minimo_de_cotistas]
  cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()
  
  fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')  
  cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]
  
  if classe == 'multimercado':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Multimercado']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'acoes':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Ações']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'rendafixa':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Renda Fixa']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'cambial':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Cambial']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
  
  #melhores
  melhores = pd.DataFrame()
  melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:top] - 1) * 100
  for cnpj in melhores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    melhores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    melhores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]

  #piores
  piores = pd.DataFrame()
  piores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=True)[:top] - 1) * 100
  for cnpj in piores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    piores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    piores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    piores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]
  
  return melhores, piores

In [5]:
def consulta_fundo(informes, cnpj):  
  fundo = informes[informes['CNPJ_FUNDO'] == cnpj].copy()
  fundo.set_index('DT_COMPTC', inplace=True)
  fundo['cotas_normalizadas'] = fundo['VL_QUOTA'] / fundo['VL_QUOTA'].iloc[0]
  return fundo

###Buscando Dados da CVM

In [6]:
cadastro = busca_cadastro_cvm()

In [7]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2020-01', data_fim='2020-12')

In [8]:
cadastro

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,DT_INI_EXERC,DT_FIM_EXERC,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,TRIB_LPRAZO,INVEST_QUALIF,TAXA_PERFM,INF_TAXA_PERFM,TAXA_ADM,INF_TAXA_ADM,VL_PATRIM_LIQ,DT_PATRIM_LIQ,DIRETOR,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
0,36.017.639/0001-27,051 7010 FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO MULTIMERCADO CP INVEST NO EXTERIOR,2020-06-05,2020-01-06,NaN,FASE PRÉ-OPERACIONAL,2020-01-06,NaN,NaN,NaN,Fundo Multimercado,2020-01-06,DI de um dia,Fechado,S,N,S,S,20.0,NaN,0.40,NaN,NaN,NaN,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A
1,34.172.417/0001-53,051 AÇÕES FUNDO DE INVESTIMENTO DE AÇÕES,2019-08-22,2019-08-20,NaN,EM FUNCIONAMENTO NORMAL,2019-08-22,2019-08-22,2020-07-01,2021-06-30,Fundo de Ações,2019-08-20,NaN,Aberto,N,N,NaN,S,0.0,NaN,0.10,NaN,1.000569e+08,2020-11-03,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
2,36.016.411/0001-12,051 ALLOCATION FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CREDITO PRIVADO,2020-04-03,2020-04-02,NaN,EM FUNCIONAMENTO NORMAL,2020-06-16,2020-06-16,2020-07-01,2021-06-30,Fundo Multimercado,2020-04-02,NaN,Aberto,S,N,S,S,0.0,NaN,0.10,NaN,1.400233e+07,2020-11-03,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
3,36.498.260/0001-86,051 ALOCAÇÃO AÇÕES FUNDO DE INVESTIMENTO DE AÇÕES,2020-04-02,2020-04-02,NaN,FASE PRÉ-OPERACIONAL,2020-04-02,NaN,NaN,NaN,Fundo de Ações,2020-07-06,NaN,Aberto,N,N,NaN,S,0.0,NaN,0.20,NaN,NaN,NaN,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
4,36.498.567/0001-87,051 ALOCAÇÃO MODERADO FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO,2020-04-13,2020-04-13,NaN,EM FUNCIONAMENTO NORMAL,2020-06-16,2020-06-16,2020-10-01,2021-09-30,Fundo Multimercado,2020-04-13,NaN,Aberto,S,N,S,S,10.0,O fundo cobrara 10.00 % do que exceder 100.00 % o indice CDIE. A taxa de performance sera paga no(s) mes(es) de Janeiro e Julho,0.70,NaN,2.393387e+06,2020-11-03,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39066,27.389.638/0001-12,ZURICH SCHRODER GROWTH FUNDO DE INVESTIMENTO MULTIMERCADO PREVIDENCIÁRIO,2018-08-07,2017-01-21,NaN,FASE PRÉ-OPERACIONAL,2017-01-21,NaN,NaN,NaN,Fundo Multimercado,2017-01-21,OUTROS,Aberto,N,S,N,S,0.0,NaN,1.00,NaN,NaN,NaN,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORE

In [9]:
informes

,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,TP_FUNDO
0,00.017.024/0001-53,2020-01-02,1132491.66,27.225023,1123583.00,0.0,0.0,1,NaN
1,00.017.024/0001-53,2020-01-03,1132685.12,27.224496,1123561.25,0.0,0.0,1,NaN
2,00.017.024/0001-53,2020-01-06,1132881.43,27.225564,1123605.31,0.0,0.0,1,NaN
3,00.017.024/0001-53,2020-01-07,1133076.85,27.226701,1123652.24,0.0,0.0,1,NaN
4,00.017.024/0001-53,2020-01-08,1132948.59,27.227816,1123698.26,0.0,0.0,1,NaN
...,...,...,...,...,...,...,...,...,...
4491238,97.929.213/0001-34,2020-12-24,70707449.04,9.302694,70694161.72,0.0,0.0,2,FI
4491239,97.929.213/0001-34,2020-12-28,70931676.80,9.332148,70917993.06,0.0,0.0,2,FI
4491240,97.929.213/0001-34,2020-12-29,69872876.85,9.339427,70973306.23,0.0,0.0,2,FI
4491241,97.929.213/0001-34,2020-12-30,70145454.01,9.360174,71130974.38,0.0,0.0,2,FI


###Fundos de Investimento: 10 Melhores e 10 Piores de 2020

In [24]:
melhores, piores = melhores_e_piores(informes, cadastro, top=8, minimo_de_cotistas=100)

In [25]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
30.282.418/0001-08,162.402084,STARBOARD SPECIAL SITUATIONS II ADVISORY FIC FIM - CP,Fundo Multimercado,1.506690e+08
13.199.100/0001-30,102.383775,SANTANDER FUNDO DE INVESTIMENTO COLABORADORES MAGAZINE LUIZA AÇÕES,Fundo de Ações,4.154515e+07
35.088.108/0001-62,93.912449,HASHDEX CRIPTOATIVOS EXPLORER FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,6.474849e+07
68.670.512/0001-07,90.851911,CSN INVEST FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,2.248356e+08
18.832.847/0001-06,76.309857,VERSA LONG BIASED FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,2.239070e+08
34.218.661/0001-00,73.164428,VITREO CANABIDIOL FUNDO DE INVESTIMENTO EM AÇÕES INVESTIMENTO NO EXTERIOR,Fundo de Ações,6.213197e+07
04.877.280/0001-71,66.945675,SANTOS CREDIT YIELD FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,4.624836e+07
30.338.659/0001-13,66.452143,CHESS ALPHA FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,2.893763e+07


In [26]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
24.837.071/0001-01,-78.449936,WM DELTA CRÉDITO ESTRUTURADO FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO - CRÉDITO PRIVADO,Fundo Multimercado,1.099282e+06
27.477.398/0001-08,-58.430956,CAPTALYS MC FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,3.529834e+07
28.254.554/0001-34,-55.350402,GESTÃO RECEBÍVEIS DE ENERGIA FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO,Fundo Multimercado,3.875225e+07
10.869.628/0001-81,-52.573737,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,2.146289e+07
10.590.125/0001-72,-52.324462,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,5.606094e+06
13.088.603/0001-39,-49.166582,CSHG DELTA ENERGIA - FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO LONGO PRAZO,Fundo Multimercado,4.985609e+08
12.987.743/0001-86,-46.376390,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I,Fundo de Ações,1.470254e+09
26.648.868/0001-96,-46.257543,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES II - BDR NÍVEL I,Fundo de Ações,3.656830e+08


###Fundos de Ações: 10 Melhores e 10 Piores de 2020

In [14]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='acoes')

In [15]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
13.199.100/0001-30,102.383775,SANTANDER FUNDO DE INVESTIMENTO COLABORADORES MAGAZINE LUIZA AÇÕES,Fundo de Ações,4.154515e+07
68.670.512/0001-07,90.851911,CSN INVEST FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,2.248356e+08
68.670.512/0001-07,90.851911,CSN INVEST FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,2.248356e+08
34.218.661/0001-00,73.164428,VITREO CANABIDIOL FUNDO DE INVESTIMENTO EM AÇÕES INVESTIMENTO NO EXTERIOR,Fundo de Ações,6.213197e+07
04.889.781/0001-78,64.466503,SANTANDER FI VALE 4 AÇÕES,Fundo de Ações,1.454054e+08
04.882.617/0001-39,64.242323,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES VALE,Fundo de Ações,2.911647e+08
04.892.107/0001-42,64.229894,BRADESCO H FUNDO DE INVESTIMENTO EM AÇÕES VALE DO RIO DOCE,Fundo de Ações,6.927134e+07
19.831.126/0001-36,63.742076,WESTERN ASSET FUNDO DE INVESTIMENTO AÇÕES BDR NÍVEL I,Fundo de Ações,2.122821e+09
04.895.210/0001-46,63.696735,SANTANDER FUNDO DE INVESTIMENTO VALE AÇÕES,Fundo de Ações,3.351806e+07


In [16]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
10.869.628/0001-81,-52.573737,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,2.146289e+07
10.590.125/0001-72,-52.324462,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,5.606094e+06
12.987.743/0001-86,-46.376390,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I,Fundo de Ações,1.470254e+09
26.648.868/0001-96,-46.257543,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES II - BDR NÍVEL I,Fundo de Ações,3.656830e+08
10.292.302/0001-34,-38.309271,HAYP FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,9.746371e+07
32.319.324/0001-83,-28.051424,TREND IBOVESPA DOLAR FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,1.213535e+07
10.551.375/0001-01,-27.179063,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA CONSTRUÇÃO CIVIL,Fundo de Ações,4.432590e+08
09.648.050/0001-54,-26.795641,BB AÇÕES CONSTRUÇÃO CIVIL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo de Ações,3.370921e+08
09.134.614/0001-30,-25.904025,BB AÇÕES BB FUNDO DE INVESTIMENTO,Fundo de Ações,3.212650e+08


###Fundos de Renda Fixa: 10 Melhores e 10 Piores de 2020

In [17]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='rendafixa')

In [18]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
04.877.280/0001-71,66.945675,SANTOS CREDIT YIELD FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,4.624836e+07
05.736.300/0001-57,51.012128,MAXI MONEY LQ FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,5.156305e+05
30.606.402/0001-03,37.480713,BB RENDA FIXA LONGO PRAZO BONDS GLOBAIS FX PRIVATE CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR FIC FI,Fundo de Renda Fixa,7.482324e+07
26.751.304/0001-84,35.548633,BB RENDA FIXA LP CRÉDITO PRIVADO FX BONDS BRASILEIROS INVESTIMENTO NO EXTERIOR PRIVATE FIC FI,Fundo de Renda Fixa,4.282483e+07
00.360.293/0001-18,34.559269,BB RENDA FIXA DIVIDA EXTERNA MIL FUNDO INVESTIMENTO,Fundo de Renda Fixa,3.137454e+08
06.114.630/0001-73,33.308248,SANTOS IV LQ FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,2.317873e+06
04.240.128/0001-83,15.494798,BV CRÉDITO ATIVO RENDA FIXA CRÉDITO PRIVADO LONGO PRAZO FUNDO DE INVESTIMENTO,Fundo de Renda Fixa,6.048290e+07
25.097.707/0001-99,9.931531,VIC SUPRA FUNDO DE INVESTIMENTO RENDA FIXA CREDITO PRIVADO INFRAESTRUTURA INCENTIVADO,Fundo de Renda Fixa,5.625179e+08
30.910.199/0001-56,9.842587,ARX ELBRUS FUNDO INCENTIVADO DE INVESTIMENTO EM INFRAESTRUTURA RENDA FIXA,Fundo de Renda Fixa,2.063442e+08


In [19]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
35.174.536/0001-08,-5.628569,ITAÚ PRIVATE RENDA FIXA VENDIDO IDKA 3 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo de Renda Fixa,9.238990e+07
02.233.665/0001-70,-4.352441,NIC FIC REFERENCIADO DI CRÉDITO PRIVADO LP,Fundo de Renda Fixa,5.561612e+06
25.274.966/0001-48,-3.774167,SELECT LIGHT FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO LONGO PRAZO -,Fundo de Renda Fixa,1.468544e+07
31.961.612/0001-74,-3.053415,SCHRODER HIGH GRADE ADVISORY FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,1.051193e+08
26.434.293/0001-09,-2.551402,GAUSS INCOME FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO RENDA FIXA - CRÉDITO PRIVADO,Fundo de Renda Fixa,1.477148e+08
01.671.480/0001-85,-2.520931,SANTOS VIRTUAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,5.928893e+06
00.840.011/0001-80,-2.095850,BTG PACTUAL YIELD DI FUNDO DE INVESTIMENTO RENDA FIXA REFERENCIADO CRÉDITO PRIVADO,Fundo de Renda Fixa,1.540397e+09
12.796.232/0001-87,-2.009130,CA INDOSUEZ AGILITÉ FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,3.767308e+08
20.057.798/0001-15,-1.638968,INTER PRIMUS FUNDO DE INVESTIMENTO RENDA FIXA CREDITO PRIVADO,Fundo de Renda Fixa,2.230690e+08


###Fundos Cambiais: 10 Melhores de 10 Piores de 2020

In [20]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='cambial')

In [21]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
04.305.193/0001-40,37.855255,BB CAMBIAL EURO LP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,1.000497e+08
04.061.044/0001-82,29.748364,BB CAMBIAL DÓLAR LP VIP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,4.735572e+08
04.128.893/0001-06,29.498190,BB CAMBIAL DOLAR LP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,2.702317e+08
01.623.535/0001-81,29.250591,ITAÚ CAMBIAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,9.755915e+08
28.516.058/0001-01,28.840355,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL ÁGORA,Fundo Cambial,1.995369e+07
23.711.486/0001-71,28.679885,OCCAM FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,5.786065e+07
02.294.024/0001-26,28.667873,BRADESCO H FUNDO DE INVESTIMENTO CAMBIAL LONGO PRAZO DÓLAR,Fundo Cambial,2.376985e+08
23.731.760/0001-74,28.625472,ITAÚ PERSONNALITÉ CAMBIAL DÓLAR PLUS FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,9.971375e+07
04.244.149/0001-77,28.574907,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL DÓLAR SPECIAL,Fundo Cambial,8.700630e+07


In [22]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
32.246.789/0001-51,-23.184920,TREND SHORT DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.352646e+07
00.819.855/0001-49,23.200132,WESTERN ASSET CÂMBIO DOLAR SILVER FICFI CAMBIAL,Fundo Cambial,2.530551e+06
01.171.956/0001-19,25.990565,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL DÓLAR,Fundo Cambial,1.201828e+07
00.819.880/0001-22,26.566017,WESTERN ASSET CÂMBIO DOLAR PRIME FICFI CAMBIAL,Fundo Cambial,3.831427e+07
00.819.889/0001-33,26.663546,WESTERN ASSET HEDGE DÓLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.046770e+08
02.543.933/0001-50,27.663559,SANTANDER FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL,Fundo Cambial,7.580689e+07
02.497.034/0001-69,27.880382,ITAÚ CAMBIAL DÓLAR - FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,3.150029e+07
05.222.509/0001-00,27.946131,BRADESCO PRIME FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL DÓLAR,Fundo Cambial,5.243433e+07
02.222.178/0001-02,27.999389,ITAÚ PERSONNALITÉ CAMBIAL DÓLAR - FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,1.169379e+08
